# Коррекция потребления электроэнергии на сезонность и температурные аномалии

Корректируем данные по потреблению в два этапа

* находим температурные аномалии 
* с учетом температурных аномалий корректируем на сезонность

Мы не даем здесь исходные данные, но приводим полный код расчетов.

## Импорт библиотек

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import io
from PIL import Image
import os

import pandas as pd
import numpy as np

from prophet import Prophet
import sqlalchemy as sa
import pyodbc

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)

import matplotlib.pyplot as plt

In [ ]:
# from https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])

## Получение данных - переписать на свой источник

In [ ]:
conn = pyodbc.connect(XXXXX)

In [ ]:
df_te = pd.read_sql(
	r"""select Date, IndexTicker, Value from IndexValue (NOLOCK) where IndexTicker like 'OES%Temp' """
	, conn
)

In [ ]:
df_te.IndexTicker = df_te.IndexTicker.str.extract(r'OES(.*)Temp',expand=False)

In [ ]:
df_te_pi = df_te.pivot(index="Date", columns = "IndexTicker", values = "Value")

In [ ]:
df_ed = pd.read_sql(
	r"""select Date, IndexTicker, Value from IndexValue (NOLOCK) where IndexTicker like 'OES%Cons'"""
	, conn
)

In [ ]:
df_ed.IndexTicker = df_ed.IndexTicker.str.extract(r'OES(.*)Cons', expand=False)
df_ed_pi = df_ed.pivot(index="Date", columns = "IndexTicker", values = "Value")

## Находим температурные аномалии

In [ ]:
def get_prophet_gap(df):
    df_gap = df.copy()
    df_gap.fillna(method='ffill',inplace=True)
    df_gap.loc[:,'ds'] = df_gap.index
    for region, data in df.items():
        df_gap.loc[:,'y'] = df_gap[region]
        m = Prophet()
        with suppress_stdout_stderr():
            m.fit(df_gap)
        future = m.make_future_dataframe(periods=365)
        forecast = m.predict(future)
        forecast.set_index('ds',inplace=True)
        df_gap.loc[:,region] =  df_gap['y'] - forecast['yhat']
    return df_gap

In [ ]:
df_te_gap = get_prophet_gap(df_te_pi)

In [ ]:
df_te_gap.head()

## Используем температурные аномалии для коррекции потребления электричества

В качестве регрессора используем оценку температурной аномалии и ее квадрат - так как экстремальные холод и жара обычно приводят к повышенному потреблению.

In [4]:
def get_prophet_gap_x_reg(df, df_ex):
    df_gap = df.copy()
    df_gap.fillna(method='ffill',inplace=True)
    df_gap.loc[:,'ds'] = df_gap.index
    for region, data in df.items():
        df_gap.loc[:,'y'] = np.log(df_gap[region])
        df_gap.loc[:,'t_gap'] = df_ex[region]
        df_gap.loc[:,'t_gap_abs'] = np.power(df_ex[region], 2)
        
        m = Prophet()
        m.add_regressor('t_gap')
        m.add_regressor('t_gap_abs')
        
        with suppress_stdout_stderr():
            m.fit(df_gap.dropna(axis=0))
            
        future = m.make_future_dataframe(periods=365)
        future['t_gap'] = 0
        future['t_gap_abs'] = 0

        forecast = m.predict(future)
        forecast.set_index('ds',inplace=True)

        df_gap[region] = (df_gap['y']-forecast['yhat'])*100
    return df_gap

In [ ]:
df_ed_gap = get_prophet_gap_x_reg(df_ed_pi, df_te_gap )